# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [1]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

# Week 1: Introduction

No code contribution expected this week

# Week 2: Camera System Modeling and Operations

We plan to
- Model the simple pinhole camera system
- Write utility functions to
    - project a 3D world point to an image
    - Compute image footprint on a surface
    - Compute the Ground Sampling Distance

## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [2]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [3]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)


## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [4]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.78   109.606]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [5]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]


In [6]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]


In [7]:
# Extra: testing compute_image_footprint_on_surface_with_angle()
# With angle of 0, the result should be the same as the result from compute_image_footprint_on_surface()

from src.camera_utils import compute_image_footprint_on_surface_with_angle
angle_x_deg = 0
angle_y_deg = 0
footprint_at_200m = compute_image_footprint_on_surface_with_angle(camera_x10, 200, angle_x_deg, angle_y_deg)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"angle_x_deg: {angle_x_deg}, angle_y_deg: {angle_y_deg}")
print(f"Footprint at 200m = {footprint_at_200m}")
print()

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

# Experiment: Increase angle_x
# Expectation: Footprint will increase
angle_x_deg = 30
angle_y_deg = 0
footprint_at_200m_ = compute_image_footprint_on_surface_with_angle(camera_x10, 200, angle_x_deg, angle_y_deg)

print(f"angle_x_deg: {angle_x_deg}, angle_y_deg: {angle_y_deg}")
print(f"Footprint at 200m = {footprint_at_200m_}")
print()

# Experiment: Increase angle_y
# Expectation: Footprint will increase
angle_x_deg = 0
angle_y_deg = 30
footprint_at_200m_ = compute_image_footprint_on_surface_with_angle(camera_x10, 200, angle_x_deg, angle_y_deg)

print(f"angle_x_deg: {angle_x_deg}, angle_y_deg: {angle_y_deg}")
print(f"Footprint at 200m = {footprint_at_200m_}")
print()

# Note: In current implementation, only one of the two angles may be non-zero.

angle_x_deg: 0, angle_y_deg: 0
Footprint at 200m = [331.75662541 248.92180476]

angle_x_deg: 30, angle_y_deg: 0
Footprint at 200m = [573.94618401 248.92180476]

angle_x_deg: 0, angle_y_deg: 30
Footprint at 200m = [331.75662541 381.08970901]



## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [8]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817469059807


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are consistent.

In [9]:
# added by CH
# Variables added by CH have suffix *_CH
# Modifying and re-using test code (from above) for project_world_point_to_image().

from src.camera_utils import reproject_image_point_to_world

distance_to_surface_CH = 50 # distance to world point (in m)
point_3d_CH = np.array([25, -30, distance_to_surface_CH], dtype=np.float32)
expected_uv_CH = np.array([6564.80, 109.60], dtype=np.float32)
uv_CH = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv_CH}")
assert np.allclose(uv_CH, expected_uv_CH, atol=1e-2)

# recovering 3d point from image using the same distance
recovered_point_3d_CH = reproject_image_point_to_world(camera_x10, distance_to_surface_CH, np.array([uv_CH[0], uv_CH[1]]))

print(f"{uv_CH} reprojected back to {recovered_point_3d_CH}")
print(f"original world point: {point_3d_CH}")
print(f"recovered world point: {recovered_point_3d_CH}")
assert np.allclose(recovered_point_3d_CH, point_3d_CH, atol=1e-2)

[ 25. -30.  50.] projected to [6564.78   109.606]
[6564.78   109.606] reprojected back to [ 25. -30.  50.]
original world point: [ 25. -30.  50.]
recovered world point: [ 25. -30.  50.]


# Week 3: Model the user requirements

For this week, we will model the dataset specifications.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned
- Scan_dimension_y: the vertical size of the rectangle to be scanned
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [10]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [11]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [12]:
# Additional check 1: Double the height, expect distance to double.
# Implication: By taking pictures from a higher height, the camera can capture a bigger area per image.
# This is useful if we want to see a zoom-out view of the scene.
# However, this will reduce the level of ground detail that the camera can capture per image.

camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_distances_ = copy.copy(expected_distances)

# Double the height. We expect the computed distance to double.
print(f"We double the height. We expected the computed distance to double.")
dataset_spec_.height = 2 * dataset_spec.height

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
expected_distances_ = 2 * expected_distances_

print(f"Computed distance at height {dataset_spec.height}: {computed_distances}")
print(f"Expected distance at height {dataset_spec.height}: {expected_distances}")
print(f"Computed distance at height {dataset_spec_.height}: {computed_distances_}")
print(f"Expected distance at height {dataset_spec_.height}: {expected_distances_}")
assert np.allclose(computed_distances_, expected_distances_, atol=1e-2)

We double the height. We expected the computed distance to double.
Computed distance at height 30.48: [15.16791291 11.38070491]
Expected distance at height 30.48: [15.17 11.38]
Computed distance at height 60.96: [30.33582583 22.76140983]
Expected distance at height 60.96: [30.34 22.76]


In [13]:
# Additional check 2: Reduce the overlap, expect the distance to increase.
# Implication: When you reduce the overlap, the camera can take pictures 
# over a larger geographical area for the same amount of available data storage.
# This is useful if we want to cover a large area with limited storage.

camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_distances_ = copy.copy(expected_distances)

print(f"Reduce overlap from 0.7 to 0.4, which is the same as changing the non-overlap ratio from 0.3 to 0.6.")
print(f"We expect the distance traveled in the horizontal direction to double.")
dataset_spec_.overlap = 0.4

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)

# We expect the horizontal distance to double, but vertical distance to be unchanged.
expected_distances_[0] = 2 * expected_distances_[0]

print(f"Computed distance with overlap {dataset_spec.overlap}: {computed_distances}")
print(f"Expected distance with overlap {dataset_spec.overlap}: {expected_distances}")
print(f"Computed distance at overlap {dataset_spec_.overlap}: {computed_distances_}")
print(f"Expected distance at overlap {dataset_spec_.overlap}: {expected_distances_}")
assert np.allclose(computed_distances_, expected_distances_, atol=1e-2)

Reduce overlap from 0.7 to 0.4, which is the same as changing the non-overlap ratio from 0.3 to 0.6.
We expect the distance traveled in the horizontal direction to double.
Computed distance with overlap 0.7: [15.16791291 11.38070491]
Expected distance with overlap 0.7: [15.17 11.38]
Computed distance at overlap 0.4: [30.33582583 11.38070491]
Expected distance at overlap 0.4: [30.34 11.38]


In [14]:
# Additional check 3: Reduce the sidelap, expect the vertical distance to increase.
# Implication: By reducing the sidelap, we can increase the vertical area that the camera can cover. 
# This is useful if we want to cover a larger geographical area for the same amount of available data storage.
# However, because we are taking picture in a lawn-mowing pattern, there will be 
# a significant time delay between adjacent images of a scene that "sidelap" each other.
# So, by reducing the sidelap, vertical motion of any object near the edge of each row
# may introduce strange image artifacts.

camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_distances_ = copy.copy(expected_distances)

print(f"Reduce sidelap from 0.7 to 0.4, which is the same as changing the non-sidelap ratio from 0.3 to 0.6.")
print(f"We expect the distance traveled in the veritical direction to double.")
dataset_spec_.sidelap = 0.4

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)

# We expect the vertical distance to double, but horizontal distance to be unchanged.
expected_distances_[1] = 2 * expected_distances[1]

print(f"Computed distance with sidelap {dataset_spec.sidelap}: {computed_distances}")
print(f"Expected distance with overlap {dataset_spec.overlap}: {expected_distances}")
print(f"Computed distance with sidelap {dataset_spec_.sidelap}: {computed_distances_}")
print(f"Expected distance with sidelap {dataset_spec_.sidelap}: {expected_distances_}")
assert np.allclose(computed_distances_, expected_distances_, atol=1e-2)

Reduce sidelap from 0.7 to 0.4, which is the same as changing the non-sidelap ratio from 0.3 to 0.6.
We expect the distance traveled in the veritical direction to double.
Computed distance with sidelap 0.7: [15.16791291 11.38070491]
Expected distance with overlap 0.7: [15.17 11.38]
Computed distance with sidelap 0.4: [15.16791291 22.76140983]
Expected distance with sidelap 0.4: [15.17 22.76]


In [15]:
# Additional check 4: Half the focal length f. Expect x-distance traveled and y-distance traveled to double.
# Rearranging the Perspective Projection Equation x = f*X/Z to X = x*Z/f,
# we can see that if we cut f to half of its original value,
# then X (the horizontal distance traveled) would double.
# Similarly, Y (the vertical distance traveled) would double as well.
# Implication: The resulting effect is similar to that of increasing the height.
# Reducing the focal length is useful if we want to take picture of a larger area without
# having to increase the height.
# However, this may introduce image distortion due to the closeness of the lens,
# (since more light will have to travel through the curved portion of the lens that is far away from the optical axis).

camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_distances_ = copy.copy(expected_distances)

print(f"Divide the focal length f by 2.")
print(f"We expect the distance traveled in both directions to double.")
camera_.fx = camera_.fx / 2
camera_.fy = camera_.fy / 2

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)

# We expect the horizontal distance to double, but vertical distance to be unchanged.
expected_distances_ = 2 * expected_distances_

print(f"Computed distance with focal lengths {camera_x10.fx} {camera_x10.fy}: {computed_distances}")
print(f"Expected distance with focal lengths {camera_x10.fx} {camera_x10.fy}: {expected_distances}")
print(f"Computed distance with focal lengths {camera_.fx} {camera_.fy}: {computed_distances_}")
print(f"Expected distance with focal lengths {camera_.fx} {camera_.fy}: {expected_distances_}")
assert np.allclose(computed_distances_, expected_distances_, atol=1e-2)

Divide the focal length f by 2.
We expect the distance traveled in both directions to double.
Computed distance with focal lengths 4938.56 4936.49: [15.16791291 11.38070491]
Expected distance with focal lengths 4938.56 4936.49: [15.17 11.38]
Computed distance with focal lengths 2469.28 2468.245: [30.33582583 22.76140983]
Expected distance with focal lengths 2469.28 2468.245: [30.34 22.76]


In [16]:
# unused template
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")

Computed distance: [15.16791291 11.38070491]


## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

## Bonus work by CH
The new function written is `compute_distance_between_images_with_angle(camera: Camera, dataset_spec: DatasetSpec, angle_x_deg: float, angle_y_deg: float)`.
                                               
In the function compute_distance_between_images_with_angle(), we added two additional angles, for the camera angles sweeping
from the z-axis toward either the x-axis or the y-axis. (So angle_x_deg is the angle that sweeps from the z-axis toward the x-axis.) 

### Assumptions:
* For the camera angle, we require `FOV/2 + abs(camera_angle) < 90 degrees`, otherwise an edge of the FOV will be at or above the horizon. (FOV = Field of View)
* The camera angle will not be so big as to cause a significant distortion to the shape of the footprint. In particular, **we assume the footprint will remain rectangularly shaped**, and the camera angle will only affect the footprint's lengths parallel to either the x or y axis, but the lengths of the remaining two sides will remain relatively unchanged.
* Only one of the angle_x_deg and angle_y_deg may be non-zero. If both angles are non-zero, then an exception will be raised when the footprint coordinates are calculated. 
* The only adjustment for the distance calculation is the footprint. We do not need to make adjustment for the overlap or sidelap.

The following is the calculation for the horizontal footprint as a function of camera angle in the x-direction (angle_x_deg) and the field of view in the x-direction (FOV_x). The calculation for the vertical footprint as a function of angle_y_deg is analogous.

If camera_angle_x >= 0, We have two cases:
* If camera_angle_x >= FOV_x / 2, then `horizontal_footprint = (tan(camera_angle_x + FOV_x/2) - tan(camera_angle_x - FOV_x/2)) * height`
* If 0 <= camera_angle_x < FOV_x/2, then `horizontal_footprint = (tan(camera_angle_x + FOV_x/2) + tan(FOV_x/2 - camera_angle_x)) * height`
![camera angle 1](assets/CH_assets/CH_camera_angle_1.png) ![camera angle 2](assets/CH_assets/CH_camera_angle_2.png)

But tan() is an odd function. By using this fact, we know that tan(FOV_x/2 - camera_angle_x) is the same as - tan(camera_angle_x - FOV_x/2), 
which means that the two formulas above are the same. 

Furthermore, by using the fact that tan() is an odd function again, we know that when camera angle_x < 0, we still get the same equations.

So either of the above equations would work whenever camera_angle_x satisfies the condition that FOV_x/2 + abs(camera_angle_x) < 90 degrees.

### Conclusion (after a slight rearrangement):
* Assuming `FOV_x/2 + abs(camera_angle_x) < 90 degrees`, then the footprint for the horizontal direction is

    ```horizontal_footprint = (tan(camera_angle_x + FOV_x / 2) - tan(camera_angle_x - FOV_x / 2)) * height```

* We then multiply the horizontal footprint by (1-overlap) to get the horizobntal distance traveled by the camera.

    ```horizontal_distance = (1 - overlap) * horizontal_footprint```

* The vertical distance traveled by the camera can be calculated similarly by calculating the vertical footprint and multiplying by (1-sidelap).

    ```vertical_footprint = (tan(camera_angle_y + FOV_y / 2) - tan(camera_angle_y - FOV_y / 2)) * height```

    ```vertical_distance = (1 - sidelap) * vertical_footprint```

In [17]:
# Print the distance traveled based on the camera angle.
# Expectation: If we are able keep the overlap constant while 
# tilting the camera_angle_x away from 0 degrees,
# then this means that the camera must have traveled a greater distance.
# Also, as the camera_angle tilts farther away from 0 degrees (assuming overlap remains constant)
# the distance tranvelled would increase at a faster rate.
# So the results below fit the expectations.
from src.plan_computation import compute_distance_between_images_with_angle
from src.camera_utils import fov

# Print FOV of the camera
[fov_deg_x, fov_deg_y] = fov(camera_x10)
print(f"Field of View FOV(degrees) [{fov_deg_x}, {fov_deg_y}]")

# Check the function's results if camera angles are (0, 0).
expected_distances_ = np.array([15.17, 11.38], dtype=np.float32) # for camera angle = 0 degrees
computed_distances_ = compute_distance_between_images_with_angle(camera_x10, dataset_spec, 0, 0)
print(f"Expected distance for camera_angle of (0, 0): {expected_distances_}")
print(f"Computed distance for camera_angle of (0, 0): {computed_distances_}")
assert np.allclose(computed_distances_, expected_distances_, atol=1e-2)

print("\nNote: To prevent the edge of FOV from going above the horizon, we require FOV/2 + abs(camera_angle) < 90 degrees.\n")

print("Adjusting camera_angle_x only:")
print(f"Camera_angle_x(deg)\tAngle(deg): fov_deg_x/2 + abs(camera_angle_x)\tComputed distance for X10 camera (meters)")
for deg in range(-45, 46, 15):
# for deg in range(-50,51, 1): # use this range to see detailed results
    computed_distances_ = compute_distance_between_images_with_angle(camera_x10, dataset_spec, deg, 0)
    print(f"{deg:>5}\t\t\t{fov_deg_x/2 + abs(deg)}\t\t\t\t[{computed_distances_[0]}\t{computed_distances_[1]}]")

print()
print("Adjusting camera_angle_y only:")
print(f"Camera_angle_y(deg)\tAngle(deg): fov_deg_y/2 + abs(camera_angle_y)\tComputed distance for X10 camera (meters)")
for deg in range(-45, 46, 15):
# for deg in range(-58,59, 1): # use this range to see detailed results
    computed_distances_ = compute_distance_between_images_with_angle(camera_x10, dataset_spec, 0, deg)
    print(f"{deg:>5}\t\t\t{fov_deg_y/2 + abs(deg)}\t\t\t\t[{computed_distances_[0]}\t{computed_distances_[1]}]")
print()

# Note: In current implementation, only one of the two angles may be non-zero.

Field of View FOV(degrees) [79.34405165142869, 63.78838119894017]
Expected distance for camera_angle of (0, 0): [15.17 11.38]
Computed distance for camera_angle of (0, 0): [15.16791291 11.38070491]

Note: To prevent the edge of FOV from going above the horizon, we require FOV/2 + abs(camera_angle) < 90 degrees.

Adjusting camera_angle_x only:
Camera_angle_x(deg)	Angle(deg): fov_deg_x/2 + abs(camera_angle_x)	Computed distance for X10 camera (meters)
  -45			84.67202582571434				[97.19605705810181	11.380704913815286]
  -30			69.67202582571434				[26.240819533127922	11.380704913815286]
  -15			54.67202582571434				[17.1015359586195	11.380704913815286]
    0			39.67202582571434				[15.167912913885834	11.380704913815286]
   15			54.67202582571434				[17.1015359586195	11.380704913815286]
   30			69.67202582571434				[26.240819533127922	11.380704913815286]
   45			84.67202582571434				[97.19605705810181	11.380704913815286]

Adjusting camera_angle_y only:
Camera_angle_y(deg)	Angle(deg): fov_de

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [18]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

# print(f"Computed speed during photo captures: {computed_speed:.2f}")
print(f"Computed speed during photo captures: {computed_speed:.4f}") # print 4 decimal places for more precision

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.0859


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [19]:
# Additional check 1: Double the allowed_movement_px, expect the speed to double.
# Implication: If we can tolerate more motion blur in the captured images,
# then the camera can fly faster.
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_speed_ = copy.copy(expected_speed)

# Set up 
allowed_movement_px_ = 2 # We allow a maximum movement of 2 px instead of 1 px

# compute
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_, allowed_movement_px_)

# We expect the allowed speed to double
expected_speed_ = 2 * expected_speed_

print(f"Double the allowed movement from 1px to 2px.")
print(f"We expect the speed to double.")
print(f"New allowed movement (px): {allowed_movement_px_}")
# print(f"Computed distance: {computed_speed_:.4f}")
print(f"Computed speed: {computed_speed_:.4f}")
print(f"Expected speed (meters/sec): {expected_speed_:.4f}")
assert np.allclose(computed_speed_, expected_speed_, atol=1e-2)

Double the allowed movement from 1px to 2px.
We expect the speed to double.
New allowed movement (px): 2
Computed speed: 6.1718
Expected speed (meters/sec): 6.1800


In [20]:
# Additional check 2: Double the exposure time, expect the speed to be halved.
# Implication: In low light situation, the camera would need a longer exposure time
# to capture each image, so the camera should fly slower.
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
expected_speed_ = copy.copy(expected_speed)

# Set up
dataset_spec_.exposure_time_ms = 2 * dataset_spec_.exposure_time_ms # We double the exposure time

# Compute
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)

# We expect the allowed speed to double
expected_speed_ = expected_speed_ / 2.0

print(f"Double the exposure time from 2ms to 4ms.")
print(f"We expect the speed to be halved.")
print(f"Original exposure time (ms): {dataset_spec.exposure_time_ms}")
print(f"New exposure time (ms): {dataset_spec_.exposure_time_ms}")
print(f"Computed speed: {computed_speed_:.4f}")
print(f"Expected speed (meters/sec): {expected_speed_:.4f}")
assert np.allclose(computed_speed_, expected_speed_, atol=1e-2)

Double the exposure time from 2ms to 4ms.
We expect the speed to be halved.
Original exposure time (ms): 2
New exposure time (ms): 4
Computed speed: 1.5430
Expected speed (meters/sec): 1.5450


In [21]:
# unused template
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed distance: {computed_speed_:.2f}")

Computed distance: 3.09


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [22]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 165 waypoints


In [23]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:20]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(x=0.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 1: Waypoint(x=15.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 2: Waypoint(x=30.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 3: Waypoint(x=45.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 4: Waypoint(x=60.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 5: Waypoint(x=75.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 6: Waypoint(x=90.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 7: Waypoint(x=105.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 8: Waypoint(x=120.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 9: Waypoint(x=

## Bonus work by CH: Generate the full flight plan in the general case with non-nadir camera angles

The waypoints are adjusted so that the center of the photos would coincide with the original waypoints only when both camera gimbal angles are zero.


In [24]:
# Testing Bonus: Generate full flight plan in the general case with non-nadir camera angles
# with the function generate_photo_plan_on_grid() in src.plan_computation
# Experiment: Increase camera gimbal x and y.
# Expectation: As we increase the camera gimbal angle in each direction, 
# the photo at each waypoint will be able to capture a larger area in the corresponding direction. 
# As a result, each row of the lawnmower pattern of the flight plan will require fewer waypoints, 
# and the total flight plan will have fewer waypoints.
    
# Detailed Waypoint printing
MAX_NUM_WAYPOINTS_TO_PRINT = 20

# dataset_spec
print(f"Scan dimension:\t[x:{dataset_spec.scan_dimension_x}\ty:{dataset_spec.scan_dimension_y}]")
print()

print("Flight Plan for various camera gimbal angles.")
angle_y = 0
for angle_x in range(0, 51, 25):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y) 
    print(f"Number of waypoints: {len(computed_plan_)}")        
    for idx, waypoint in enumerate(computed_plan_[:MAX_NUM_WAYPOINTS_TO_PRINT]):
        print(f"Idx {idx}: {waypoint}")
    if len(computed_plan_) >= MAX_NUM_WAYPOINTS_TO_PRINT:
        print("...")
    print()            

angle_x = 0    
for angle_y in range(0, 51, 25):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y) 
    print(f"Number of waypoints: {len(computed_plan_)}")        
    for idx, waypoint in enumerate(computed_plan_[:MAX_NUM_WAYPOINTS_TO_PRINT]):
        print(f"Idx {idx}: {waypoint}")
    if len(computed_plan_) >= MAX_NUM_WAYPOINTS_TO_PRINT:
        print("...")
    print()            
    


Scan dimension:	[x:150	y:150]

Flight Plan for various camera gimbal angles.
Camera Angles: [Angle_x:0	Angle_y:0]
Number of waypoints: 165
Idx 0: Waypoint(x=0.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 1: Waypoint(x=15.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 2: Waypoint(x=30.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 3: Waypoint(x=45.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 4: Waypoint(x=60.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 5: Waypoint(x=75.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 6: Waypoint(x=90.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angle_y_deg=0)
Idx 7: Waypoint(x=105.0, y=0.0, z=30.48, speed=3.085919782284715, camera_angle_x_deg=0, camera_angl

## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal speed profile

## Bonus work by CH: Time computation

### Assumption
We assume the drone can change direction without loss of speed, such as at the end of each row of the lawnmower flight pattern.

### Variables

We define the variables as follows:
* acceleration $a = 3.5 m/s^2$
* $v_{max} = 16 m/s$
* $v_1$ = speed at waypoint
* $D$ = distance between waypoints
* $T$ = time between waypoints

We also assume that max deceleration = - max acceleration.

### Case 1: Assume the distance $D$ between waypoints is far enought so $v_{max}$ is attained
In this case, we can breakdown the calculation into three sections:
* accelerating from $v_1$ to $v_{max}$, for a distance of $d_1$ with time $t_1$
* travelling with speed of $v_{max}$, for a distance of $d_2$ with  time $t_2$
* decelerating from $v_{max}$ to $v_1$, for a distance of $d_3$ with time $t_3$

#### Calculation of distance and time with constant acceleration from a speed of $v_1$ to a speed of $v_{max}$.
Let 
$t_1$ = time during acceleration, 
and 
$\Delta v = v_{max} - v_1$.

Since $a * t_1 = \Delta v$, so $t_1 = \Delta v / a$. 

Also by basic kinematics for constant acceleration, $d_1 = v_1  t_1 + (1/2) a t_1^2$

After substitution, we get $d_1 = v_1 \Delta v / a + (1/2) a (\Delta v / a)^2 = v_1 \Delta v / a + (1/2) (\Delta v)^2 / a$

By symmetry, during deceleration from $v_{max}$ to $v_1$, we would have $d_3 = d_1$ with time $t_3 = t_1$.

Since $D = d_1 + d_2 + d_3$, the distance while travelling with 
a speed of $v_{max}$ would be $d_2 = D - 2 d_1$ with a time $t_2 = d_2 / v_{max}$

So the total time between waypoints would be $T = t_1 + t_2 + t_3$

### Case 2: Assume the distance $D$ between waypoints is less than $d_1 + d_3$ as calculated in Case 1
In this case, $v_{max}$ is not attained, but instead the speed would only increase up to $v_2$,
and the calculation would only include two sections:
* acceleration from $v_1$ to $v_2$, for a distance of $D/2$ with time $t_a$
* deceleration from $v_2$ to $v_1$, for a distance of $D/2$ with time $t_b$

where $t_a = t_b$ by symmetry.

#### Calculation of time with constant acceleration from a speed of $v_1$ to a speed of $v_2$.
Let 
$t_a$ = time during acceleration, 
and 
$\Delta v_a = v_2 - v_1$.

Since $a t_a = \Delta v_a$, so $t_a = \Delta v_a / a$. 

By basic kinematics, $D/2 = v_1 t_a + (1/2) a t_a^2$

After substitution, we have $D/2 = v_1 (\Delta v_a / a) + (1/2) a (\Delta v_a / a)^2 = v_1 (v_2 - v_1) / a + (1/2) (v_2 - v_1)^2 / a$

After simplification, we get a maximum speed of $v_2 = \sqrt{aD + v_1^2}$

So $t_a = (v_2 - v_1) / a = (\sqrt{aD + v_1^2} - v_1) / a$ 

Therefore the total time between waypoints would be $T = t_a + t_b = 2 (\sqrt{aD + v_1^2} - v_1) / a$

In [25]:
# Testing Bonus: Time Computation for the function time_between_waypoints() from src.plan_computation
# Experiment: Increase the camera gimbal angle.
# Expectation: The distance between waypoints will increase, and flight time between consecutive waypoints will increase.

print(f"Testing time_between_waypoints() from src.plan_computation")
print()

# Drone Speed and acceleration limit
max_speed = 16
max_acc = 3.5

# calculating travel time
from src.plan_computation import time_between_waypoints
import math

# Detailed Waypoint printing
MAX_NUM_WAYPOINTS_TO_PRINT = 2
# Indices of Waypoints to calculate travel time for
# Calculate travel distance and travel time between the first two waypoints.
i = 0
j = 1

# initialize the angles
angle_x = 0
angle_y = 0

for angle_x in range(0, 31, 15):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y)
    print(f"Number of waypoints: {len(computed_plan_)+1}")
    distance = math.sqrt((computed_plan_[i].x - computed_plan_[j].x) ** 2 
                         + (computed_plan_[i].y - computed_plan_[j].y) ** 2) # Euclidean distance
    time = time_between_waypoints(computed_plan_[i], computed_plan_[j], max_speed, max_acc)
    print(f"Between Waypoint[{i}] and Waypoint[{j}]: ")
    print(f"\tDistance:{distance:0.4f}\tTime:{time:0.4f}")
    print()
    print(50 * "-") # section separator

# reinitialize the angles
angle_x = 0
angle_y = 0

for angle_y in range(0, 31, 15):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y)
    print(f"Number of waypoints: {len(computed_plan_)+1}")
    distance = math.sqrt((computed_plan_[i].x - computed_plan_[j].x) ** 2 
                         + (computed_plan_[i].y - computed_plan_[j].y) ** 2) # Euclidean distance
    time = time_between_waypoints(computed_plan_[i], computed_plan_[j], max_speed, max_acc)
    print(f"Between Waypoint[{i}] and Waypoint[{j}]: ")
    print(f"\tDistance:{distance:0.4f}\tTime:{time:0.4f}")
    print()
    print(50 * "-") # section separator



Testing time_between_waypoints() from src.plan_computation

Camera Angles: [Angle_x:0	Angle_y:0]
Number of waypoints: 166
Between Waypoint[0] and Waypoint[1]: 
	Distance:15.0000	Time:2.7369

--------------------------------------------------
Camera Angles: [Angle_x:15	Angle_y:0]
Number of waypoints: 151
Between Waypoint[0] and Waypoint[1]: 
	Distance:16.6667	Time:2.9438

--------------------------------------------------
Camera Angles: [Angle_x:30	Angle_y:0]
Number of waypoints: 106
Between Waypoint[0] and Waypoint[1]: 
	Distance:25.0000	Time:3.8652

--------------------------------------------------
Camera Angles: [Angle_x:0	Angle_y:0]
Number of waypoints: 166
Between Waypoint[0] and Waypoint[1]: 
	Distance:15.0000	Time:2.7369

--------------------------------------------------
Camera Angles: [Angle_x:0	Angle_y:15]
Number of waypoints: 144
Between Waypoint[0] and Waypoint[1]: 
	Distance:15.0000	Time:2.7369

--------------------------------------------------
Camera Angles: [Angle_x:0	A

In [26]:
# Testing Bonus: Time computation for the function total_time_for_flightplan() in src.plan_computation
# Experiment: We increase the camera's gimbal angles.
# Expection: The number of waypoints will decrease while the distance between consecutive waypoints will increase.
# Althought the drone will take longer to travel between consecutive waypoints, the drone can fly faster between waypoints.
# Consequently, the total time for the whole flightplan will decrease because the total distance is fixed.

from src.plan_computation import time_between_waypoints, total_time_for_flightplan

max_speed = 16
max_acc = 3.5

print("Number of Waypoints and Total Flight Time for various camera gimbal angles.")
print()

# initialize the angles
angle_x = 0
angle_y = 0

for angle_x in range(0, 31, 15):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y) 
    print(f"Number of waypoints: {len(computed_plan_)}")
    total_time = total_time_for_flightplan(computed_plan_, max_speed, max_acc)
    print(f"Total time for flightplan: {total_time:0.4f}")
    print()
print(50 * "-") # section separator

# reinitialize the angles
angle_x = 0
angle_y = 0

for angle_y in range(0, 31, 15):
    print(f"Camera Angles: [Angle_x:{angle_x}\tAngle_y:{angle_y}]")
    computed_plan_ = generate_photo_plan_on_grid(camera_x10, dataset_spec, angle_x_deg = angle_x, angle_y_deg = angle_y) 
    print(f"Number of waypoints: {len(computed_plan_)}")
    total_time = total_time_for_flightplan(computed_plan_, max_speed, max_acc)
    print(f"Total time for flightplan: {total_time:0.4f}")
    print()
print(50 * "-") # section separator


Number of Waypoints and Total Flight Time for various camera gimbal angles.

Camera Angles: [Angle_x:0	Angle_y:0]
Number of waypoints: 165
Total time for flightplan: 440.7034

Camera Angles: [Angle_x:15	Angle_y:0]
Number of waypoints: 150
Total time for flightplan: 427.5781

Camera Angles: [Angle_x:30	Angle_y:0]
Number of waypoints: 105
Total time for flightplan: 378.0393

--------------------------------------------------
Camera Angles: [Angle_x:0	Angle_y:0]
Number of waypoints: 165
Total time for flightplan: 440.7034

Camera Angles: [Angle_x:0	Angle_y:15]
Number of waypoints: 143
Total time for flightplan: 384.6831

Camera Angles: [Angle_x:0	Angle_y:30]
Number of waypoints: 110
Total time for flightplan: 300.1819

--------------------------------------------------


# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [27]:
# setup to save plotly figure
import os
if not os.path.exists("assets/CH_images"):
    os.mkdir("assets/CH_images")

In [28]:
fig = plot_photo_plan(computed_plan)
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/fig1.svg") # to save static figure

![fig1](assets/CH_images/fig1.svg)

$\color{red}{\text{TODO: }}$ Compute the following ablations (and any other you can think of). 
You need to describe the input params you are changing, what impact you can observe, explanation behind the change in output, and practical implication of the correlation.

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

In [29]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000

print(camera_, dataset_spec_)

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/fig2.svg") # to save static figure

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144) DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)


![fig2](assets/CH_images/fig2.svg)

# Flight Plan Experiments
The following is a series of flight plan experiments. It begins with the flight plan from the original flight specification given above. Each of the experiment would make one change to the original specification, and the effect on the flight plan is summarized and visualized.

In [30]:
# Photo Plan Experiment Preparation
# These are helper functions to make experiments easier

from src.visualization import plot_photo_plan_with_area
from src.plan_computation import waypoint_count

# This helper function print the current specification
def print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_=1):
    print('\tCamera: ', camera_)
    print('\tDataset_spec: ', dataset_spec_)
    print('\tCamera_angle_x_deg: ', camera_angle_x_deg_)
    print('\tCamera_angle_y_deg: ', camera_angle_y_deg_)
    print('\tallowed_movement_px: ', allowed_movement_px_)

# def create_plan_and_plot(camera_, dataset_spec_, experiment_name, figure_title, var_change, code_for_var_change):
def create_plan_and_plot(camera_, dataset_spec_, figure_title='', 
                         camera_angle_x_deg_=0, camera_angle_y_deg_=0, allowed_movement_px_=1):
    # allowed_movement_px (float, optional): The maximum allowed movement during active photo capture (in pixels). Defaults to 1 px.

    # set the features of the rectangular regions representing the photo areas
    fill_alpha = 0.01 # alpha for the fill of the photo areas
    line_alpha = 0.3 # alpha for the outline of the photo areas
    jitter_amount = 1.0 # photo area as shown is offsetted with a random jitter

    # generate photo plan for specific camera angles
    computed_plan_ = generate_photo_plan_on_grid(camera_, dataset_spec_,                                                  
                                                 angle_x_deg = camera_angle_x_deg_, 
                                                 angle_y_deg = camera_angle_y_deg_,
                                                 allowed_movement_px = allowed_movement_px_)

    # setup for the figure
    fig = plot_photo_plan_with_area(computed_plan_, camera_, dataset_spec_, figure_title, fill_alpha, line_alpha, jitter_amount)
    return computed_plan_, fig
    # end of function


## Print flight time summary
def print_plan_summary(computed_plan_, max_speed, max_acc):
    print("PLAN SUMMARY")
    print(f"Camera angles x,y: [{computed_plan_[0].camera_angle_x_deg}, {computed_plan_[0].camera_angle_y_deg}]")
    number_of_waypoints, num_per_row, num_rows = waypoint_count(computed_plan_)
    print(f"Number of waypoints: {number_of_waypoints}")        
    print(f"Number of waypoints per row: {num_per_row}")
    print(f"Number of rows of waypoints: {num_rows}")    
    print(f"Speed at waypoint: {computed_plan_[0].speed:0.4f}")
    total_time = total_time_for_flightplan(computed_plan_, max_speed, max_acc)
    print(f"Total time for flightplan: {total_time:0.4f}")


## helper function for running experiments
def run_experiment(camera_, dataset_spec_, 
                   camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                   max_speed, max_acc,
                   experiment_name, experiment_note, figure_title):
    # Begin Experiment
    print("-"*50)
    print(experiment_name)
    print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_)
    print(experiment_note)
    print()
        
    # run experiment
    computed_plan_, fig = create_plan_and_plot(camera_, dataset_spec_, figure_title, 
                                               camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_)
    
    ##############
    # Show results
    
    # Plan Summary
    ### max_speed = 16
    ### max_acc = 3.5
    print("-"*50)
    print("Experiment Result:")
    print_plan_summary(computed_plan_, max_speed, max_acc)
    
    return computed_plan_, fig


# Experiment 0: Flight plan with the original specification

In [31]:
# Experiment 0: Flight plan with the original specification
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 0: Flight plan with the original specification"
experiment_note = f"This is the original setup before making modification."
# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'None' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

## For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Keep result for later comparison

# Keep a copy of the result of the original computed_plan_
computed_plan_original_ = computed_plan_

##########################
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp0.svg") # to save static figure


Modifications:
Code modification: None

--------------------------------------------------
Experiment 0: Flight plan with the original specification
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
This is the original setup before making modification.

--------------------------------------------------
Experiment Result:
PLAN SUMMARY
Camera angles x,y: [0, 0]
Number of waypoints: 165
Number of waypoints per row: 11
Number of rows of waypoints: 15
Speed at waypoint: 3.0859
Total time for flightplan: 440.7034


![exp0](assets/CH_images/exp0.svg)

# Experiment 1: Reduce overlap from 0.7 to 0.4

In [32]:
# Experiment 1: Reduce overlap from 0.7 to 0.4
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 1: Reduce overlap from 0.7 to 0.4"
experiment_note = f'''NOTE:
Reduce the overlap, expect the distance to increase.
Implication: When you reduce the overlap, the camera can take pictures 
over a larger geographical area for the same amount of available data storage.

This is useful if we want to cover a large area with limited storage.

Expectation: Reducing overlap should result in the the following:
Reduction of:
    Total number of waypoints
    Number of waypoints per row
    Total flight time'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'dataset_spec_.overlap = 0.4' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp1.svg") # to save static figure


Modifications:
Code modification: dataset_spec_.overlap = 0.4

--------------------------------------------------
Experiment 1: Reduce overlap from 0.7 to 0.4
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.4, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Reduce the overlap, expect the distance to increase.
Implication: When you reduce the overlap, the camera can take pictures 
over a larger geographical area for the same amount of available data storage.

This is useful if we want to cover a large area with limited storage.

Expectation: Reducing overlap should result in the the following:
Reduction of:
    Total number of waypoints
    Number of waypoints per row
    Total flight time

----------------------------------

![exp1](assets/CH_images/exp1.svg)

# Experiment 2: Reduce sidelap from 0.7 to 0.4

In [33]:
# Experiment 2: Reduce sidelap from 0.7 to 0.4
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 2: Reduce sidelap from 0.7 to 0.4"
experiment_note = f'''Note:
Reduce the sidelap, expect the vertical distance to increase.
Implication: By reducing the sidelap, we can increase the vertical area that the camera can cover. 

This is useful if we want to cover a larger geographical area for the same amount of available data storage.
However, because we are taking picture in a lawn-mowing pattern, there will be 
a significant time delay between adjacent images of a scene that "sidelap" each other.
So, by reducing the sidelap, vertical motion of any object near the edge of each row
may introduce strange image artifacts.

Expectation: Reducing sidelap should result in the the following:
Reduction of:
    Total number of waypoints
    Number of rows
    Total flight time'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'dataset_spec_.sidelap = 0.4' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp2.svg") # to save static figure

Modifications:
Code modification: dataset_spec_.sidelap = 0.4

--------------------------------------------------
Experiment 2: Reduce sidelap from 0.7 to 0.4
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.4, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
Note:
Reduce the sidelap, expect the vertical distance to increase.
Implication: By reducing the sidelap, we can increase the vertical area that the camera can cover. 

This is useful if we want to cover a larger geographical area for the same amount of available data storage.
However, because we are taking picture in a lawn-mowing pattern, there will be 
a significant time delay between adjacent images of a scene that "sidelap" each other.
So, by reducing the sidelap, vertica

![exp2](assets/CH_images/exp2.svg)

# Experiment 3: Double the height of the scan from 30.48 to 60.96

In [34]:
# Experiment 3: Double the height of the scan from 30.48 to 60.96
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 3: Double the height of the scan from 30.48 to 60.96"
experiment_note = f'''NOTE:
Double the height, expect distance to double.
Implication: By taking pictures from a higher height, the camera can capture a bigger area per image.

This is useful if we want to see a zoom-out view of the scene.
However, this will reduce the level of ground detail that the camera can capture per image.

Expectation: Increasing the height should result in the the following:
Reduction of:
    Total number of waypoints
    Number of waypoints per row
    Number of rows
    Total flight time'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'dataset_spec_.height = 2 * dataset_spec_.height' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp3.svg") # to save static figure



Modifications:
Code modification: dataset_spec_.height = 2 * dataset_spec_.height

--------------------------------------------------
Experiment 3: Double the height of the scan from 30.48 to 60.96
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=60.96, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Double the height, expect distance to double.
Implication: By taking pictures from a higher height, the camera can capture a bigger area per image.

This is useful if we want to see a zoom-out view of the scene.
However, this will reduce the level of ground detail that the camera can capture per image.

Expectation: Increasing the height should result in the the following:
Reduction of:
    Total number of waypoints
    Number of wa

![exp3](assets/CH_images/exp3.svg)

# Experiment 4: Halving the focal length from 4938.56 to 2469.28

In [35]:
# Experiment 4: Halving the focal length from 4938.56 to 2469.28
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 4: Halving the focal length from 4938.56 to 2469.28"
experiment_note = f'''NOTE:
Halving the focal length f. Expect x-distance traveled and y-distance traveled to double.
Implication: The resulting effect is similar to that of increasing the height.

This is useful if we want to take picture of a larger area without
having to increase the height.
However, this may introduce image distortion due to the closeness of the lens,
(since more light will have to travel through the curved portion of the lens that is far away from the optical axis).

Expectation: Reducing the focal length of the camera should result in the the following:
Reduction of:
    Total number of waypoints
    Number of waypoints per row
    Number of rows
    Total flight time
'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = '(camera_.fx, camera_.fy)  = (camera_.fx / 2, camera_.fy / 2)' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp4.svg") # to save static figure


Modifications:
Code modification: (camera_.fx, camera_.fy)  = (camera_.fx / 2, camera_.fy / 2)

--------------------------------------------------
Experiment 4: Halving the focal length from 4938.56 to 2469.28
	Camera:  Camera(fx=2469.28, fy=2468.245, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Halving the focal length f. Expect x-distance traveled and y-distance traveled to double.
Implication: The resulting effect is similar to that of increasing the height.

This is useful if we want to take picture of a larger area without
having to increase the height.
However, this may introduce image distortion due to the closeness of the lens,
(since more light will have to travel through the curved portion of the lens

![exp4](assets/CH_images/exp4.svg)

# Experiment 5: Increase exposure time from 2ms to 1000ms

In [36]:
# Experiment 5: Increase exposure time from 2ms to 1000ms
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 5: Increase exposure time from 2ms to 1000ms"
experiment_note = f'''NOTE:
Increase exposure time, expect the speed to be reduced.
Implication: In low light situation, the camera would need a longer exposure time to capture each image, so the camera should fly slower.

This is useful if we want to take photos in a low light situation.

Expectation: Increasing the exposure time should result in the the following:
Reduction of:
    Speed at waypoint
Increase of:
    Total flight time'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'dataset_spec_.exposure_time_ms = 1000' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp5.svg") # to save static figure



Modifications:
Code modification: dataset_spec_.exposure_time_ms = 1000

--------------------------------------------------
Experiment 5: Increase exposure time from 2ms to 1000ms
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=1000)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Increase exposure time, expect the speed to be reduced.
Implication: In low light situation, the camera would need a longer exposure time to capture each image, so the camera should fly slower.

This is useful if we want to take photos in a low light situation.

Expectation: Increasing the exposure time should result in the the following:
Reduction of:
    Speed at waypoint
Increase of:
    Total flight time

--------------------------------------

![exp5](assets/CH_images/exp5.svg)

# Experiment 6: Increase allowed_movement_px from 1 to 2


In [37]:
# Experiment 6: Increase allowed_movement_px from 1 to 2
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 6: Increase allowed_movement_px from 1 to 2"
experiment_note = f'''NOTE:
Double the allowed_movement_px during active photo capture, expect the speed to double.
Implication: If we can tolerate more motion blur in the captured images, then the drone can fly faster.

This is useful if we want to complete the photo capturing session in a shorter time span.

Expectation: Increasing allowed_movement_px from 1 to 2 should result in the the following:
Increase of:
    Speed at waypoint
Reduction of:
    Total flight time
'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'allowed_movement_px_ = 2' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp6.svg") # to save static figure


Modifications:
Code modification: allowed_movement_px_ = 2

--------------------------------------------------
Experiment 6: Increase allowed_movement_px from 1 to 2
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  0
	allowed_movement_px:  2
NOTE:
Double the allowed_movement_px during active photo capture, expect the speed to double.
Implication: If we can tolerate more motion blur in the captured images, then the drone can fly faster.

This is useful if we want to complete the photo capturing session in a shorter time span.

Expectation: Increasing allowed_movement_px from 1 to 2 should result in the the following:
Increase of:
    Speed at waypoint
Reduction of:
    Total flight time


-------------------

![exp6](assets/CH_images/exp6.svg)

# Experiment 7A: Non-Nadir camera_angle_x_deg_ From 0 deg to 15 deg


In [38]:
# Experiment 7A: Non-Nadir camera_angle_x_deg_ From 0 deg to 15 deg
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 7A: Non-Nadir camera_angle_x_deg_ From 0 deg to 15 deg"
experiment_note = f'''NOTE:
Increase the camera gimbal in the x direction from 0 to 15 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift to the left and the footprint to elongate in the horizontal direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing session in a shorter time, assuming we can tolerate image distortion.

Expectation: Increasing camera angle_x should result in the the following:
Waypoint: Shifting to the left.
Photo area: 
    Elongation in the horizontal direction.
    The center of photo is no longer at the center of the rectangular area of the ground.
Reduction of:
    Number of waypoints
    Number of waypoints per row
    Total flight time
Increase of:
    Speed at waypoint (potentially, if angle is big enough)'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'camera_angle_x_deg_ = 15' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp7A.svg") # to save static figure



Modifications:
Code modification: camera_angle_x_deg_ = 15

--------------------------------------------------
Experiment 7A: Non-Nadir camera_angle_x_deg_ From 0 deg to 15 deg
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  15
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Increase the camera gimbal in the x direction from 0 to 15 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift to the left and the footprint to elongate in the horizontal direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturin

![exp7A](assets/CH_images/exp7A.svg)

# Experiment 7B: Non-Nadir camera_angle_x_deg_ From 0 deg to 45 deg


In [39]:
# Experiment 7B: Non-Nadir camera_angle_x_deg_ From 0 deg to 45 deg
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 7B: Non-Nadir camera_angle_x_deg_ From 0 deg to 45 deg"
experiment_note = f'''NOTE:
Increase the camera gimbal in the x direction from 0 to 45 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift to the left and the footprint to elongate in the horizontal direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing session in a shorter time, assuming we can tolerate image distortion.
However, 45 degrees may be too large so that the assumption that the footprint will remain rectangularly shaped may not be realistic.

Expectation: Increasing camera angle_x should result in the the following:
Waypoint: Shifting to the left.
Photo area: 
    Elongation in the horizontal direction.
    The center of photo is no longer at the center of the rectangular area of the ground.
Reduction of:
    Number of waypoints
    Number of waypoints per row
    Total flight time
Increase of:
    Speed at waypoint (potentially, if angle is big enough)'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'camera_angle_x_deg_ = 45' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp7B.svg") # to save static figure



Modifications:
Code modification: camera_angle_x_deg_ = 45

--------------------------------------------------
Experiment 7B: Non-Nadir camera_angle_x_deg_ From 0 deg to 45 deg
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  45
	Camera_angle_y_deg:  0
	allowed_movement_px:  1
NOTE:
Increase the camera gimbal in the x direction from 0 to 45 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift to the left and the footprint to elongate in the horizontal direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturin

![exp7B](assets/CH_images/exp7B.svg)

# Experiment 8A: Non-Nadir camera_angle_y_deg_ From 0 deg to 15 deg


In [40]:
# Experiment 8A: Non-Nadir camera_angle_y_deg_ From 0 deg to 15 deg
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 8A: Non-Nadir camera_angle_y_deg_ From 0 deg to 15 deg"
experiment_note = f'''NOTE:
Increase the camera gimbal in the y direction from 0 to 15 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift downward and the footprint to elongate in the vertical direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing session in a shorter time, assuming we can tolerate image distortion.

Expectation: Increasing camera angle_y should result in the the following:
Waypoint: Shifting downward.
Photo area: 
    Elongation in the vertical direction.
    The center of photo is no longer at the center of the rectangular area of the ground.
Reduction of:
    Number of waypoints
    Number of rows
    Total flight time
Increase of:
    Speed at waypoint (potentially, if angle is big enough)'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'camera_angle_y_deg_ = 15' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp8A.svg") # to save static figure


Modifications:
Code modification: camera_angle_y_deg_ = 15

--------------------------------------------------
Experiment 8A: Non-Nadir camera_angle_y_deg_ From 0 deg to 15 deg
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  15
	allowed_movement_px:  1
NOTE:
Increase the camera gimbal in the y direction from 0 to 15 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift downward and the footprint to elongate in the vertical direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing ses

![exp8A](assets/CH_images/exp8A.svg)

# Experiment 8B: Non-Nadir camera_angle_y_deg_ From 0 deg to 45 deg


In [41]:
# Experiment 8B: Non-Nadir camera_angle_y_deg_ From 0 deg to 45 deg
##########################
### Modify this section for running experiments.
experiment_name = f"Experiment 8B: Non-Nadir camera_angle_y_deg_ From 0 deg to 45 deg"
experiment_note = f'''NOTE:
Increase the camera gimbal in the y direction from 0 to 45 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift downward and the footprint to elongate in the vertical direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing session in a shorter time, assuming we can tolerate image distortion.
However, 45 degrees may be too large so that the assumption that the footprint will remain rectangularly shaped may not be realistic.

Expectation: Increasing camera angle_y should result in the the following:
Waypoint: Shifting downward.
Photo area: 
    Elongation in the vertical direction.
    The center of photo is no longer at the center of the rectangular area of the ground.
Reduction of:
    Number of waypoints
    Number of rows
    Total flight time
Increase of:
    Speed at waypoint (potentially, if angle is big enough)'''

# plot title 
figure_title = "Photo Plan for " + experiment_name

# Code for modifications
code_for_var_change = 'camera_angle_y_deg_ = 45' # Enter python code for variables to modify
### END MODIFICATION ###
##########################

# Recreate the original experiment condition before making modifications.
# Copy original setup
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
# Add camera angles
camera_angle_x_deg_ = 0
camera_angle_y_deg_ = 0
# Add maximum movement allowed during active photo capture to prevent motion blur
allowed_movement_px_ = 1 # Default value is 1 px
# Additional Parameters
max_speed = 16
max_acc = 3.5

### For debugging
#print("Original Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Running the code for modification of variable values
print('Modifications:')
exec(code_for_var_change)
print(f'Code modification: {code_for_var_change}')
print()

## For debugging
#print("Experimental Specifications")
#print_specifications(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_)

##########################
# Begin Experiment
computed_plan_, fig = run_experiment(camera_, dataset_spec_, camera_angle_x_deg_, camera_angle_y_deg_, allowed_movement_px_,
                                max_speed, max_acc,
                                experiment_name, experiment_note, figure_title)

##########################
# Compare with original results
print()
print("Compare with Original Result:")
print_plan_summary(computed_plan_original_, max_speed, max_acc)

##########################
# Show results
# fig.show() # uncomment this line to display an interactive plot
fig.write_image("assets/CH_images/exp8B.svg") # to save static figure



Modifications:
Code modification: camera_angle_y_deg_ = 45

--------------------------------------------------
Experiment 8B: Non-Nadir camera_angle_y_deg_ From 0 deg to 45 deg
	Camera:  Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
	Dataset_spec:  DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)
	Camera_angle_x_deg:  0
	Camera_angle_y_deg:  45
	allowed_movement_px:  1
NOTE:
Increase the camera gimbal in the y direction from 0 to 45 degrees while keepting the center of the photos at the same location.
Expect the waypoint locations to shift downward and the footprint to elongate in the vertical direction.
Implication: This will allow each photo to capture a larger area but with distortion, 
while allowing the photo capturing session to complete in shorter time. 

This is useful if we want to complete the photo capturing ses

![exp8B](assets/CH_images/exp8B.svg)

# Functions that I wrote in a unique way

## Function: compute_image_footprint_on_surface_coord() in src/camera_utils.py
In my visualization, I wanted to show the footprint corresponding to each waypoint for non-nadir camera angles. In order to do that, I created this new function to calculate coordinates of the footprint relative to the camera's position. This also calculates the coordinate on the footprint corresponding to the center of the camera's view (which will not be at the center of the footprint when the angle is non-zero).


## Function: closest_distance_to_footprint() in  src/camera/utils.py
After a discussion with Ayush, I learned that I need to consider the closest point on the footprint when I calculate the waypoint speed to avoid motion blue. So I created this function to calculate that distance. This distance will be the same as the drone height when angle is small enough so that the drone is flying over any part of the footprint. However, when the angle is big enough so that the drone is not directly over any part of the footprint, this function will calculate the shortest distance from the drone to any point on the footprint.

## Function: plot_photo_plan_with_area() in src/visualization.py
In my visualization, I wanted to overlay the rectangular regions of the photos over the plot with just the flight path with waypoints. So I created this function to add the rectangles over the original plot. It took a while to turn the transparency of the colors and color so that the visualization will not be too dark. I also used different colors to represent the footprint for different waypoints, so I use the hue-saturation-value scale so that the hues can recycle.


# Lesson Learned From This Project

In this project, I learned that the attempt at creating visualization can help uncover hidden assumptions that I have in my thinking. 

For example, when I first added the non-nadir camera angles, I assumed the footprint will remain relatively rectangular. But during the process of creating the visualization, I was forced to think through the details of the situation, and I realized the distortion to the shape of the footprint may be so large that the footprint will be far from rectangular when the angle is large. 


# Acknowledgment

Special thanks to Ayush Baid for guiding me through this project.